In [2]:
import pandas as pd
import numpy as np

In [3]:
#Let's get started by reading in the data

pitches = pd.read_csv('savant_pitch_level.csv')
pitches.head()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,role_key,sp_indicator,rp_indicator,pitch_number_appearance,pitcher_at_bat_number,times_faced
0,FF,2021-04-05,93.2,0.84,5.74,"Duffy, Danny",592696,518633,strikeout,swinging_strike,...,Standard,166.0,-0.014,-0.134,SP,1,0,97,22,3
1,SL,2021-04-05,82.9,1.03,5.69,"Duffy, Danny",592696,518633,NaN,foul,...,Standard,309.0,0.000,0.000,SP,1,0,96,22,3
2,FF,2021-04-05,94.6,-1.85,5.77,"Foltynewicz, Mike",543760,592314,caught_stealing_3b,ball,...,Standard,220.0,0.014,-0.202,SP,1,0,95,19,3
3,FF,2021-04-05,97.1,1.86,6.57,"Rodón, Carlos",657108,607074,field_out,hit_into_play,...,Standard,147.0,-0.007,-0.264,SP,1,0,95,22,3
4,FF,2021-04-05,93.6,0.60,5.85,"Duffy, Danny",592696,518633,NaN,ball,...,Standard,164.0,0.000,0.043,SP,1,0,95,22,3


In [4]:
pitches['role_key'].unique() # they have already split in SP and RP

array(['SP', 'RP'], dtype=object)

In [5]:
relievers = pitches.loc[pitches['role_key'] == 'RP', :]

In [6]:
relievers.shape

(908219, 91)

In [7]:
len(relievers['pitcher'].unique())

1259

In [8]:
sorted_relievers = relievers.sort_values(by=['game_date', 'pitcher', 'pitch_number_appearance'])

In [9]:
sorted_relievers.head()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,role_key,sp_indicator,rp_indicator,pitch_number_appearance,pitcher_at_bat_number,times_faced
17863,FF,2021-04-01,87.5,-0.33,5.67,"Petit, Yusmeiro",621043,433589,NaN,ball,...,Standard,211.0,0.000,0.093,RP,0,1,1,1,1
17758,FF,2021-04-01,87.2,-0.38,5.52,"Petit, Yusmeiro",621043,433589,NaN,ball,...,Standard,212.0,0.000,0.144,RP,0,1,2,1,1
17615,FC,2021-04-01,83.5,-0.66,5.60,"Petit, Yusmeiro",621043,433589,field_out,hit_into_play,...,Standard,194.0,0.045,-0.712,RP,0,1,3,1,1
17484,FF,2021-04-01,85.6,-2.25,5.52,"Petit, Yusmeiro",670541,433589,NaN,ball,...,Standard,209.0,0.000,0.056,RP,0,1,4,2,1
17322,CH,2021-04-01,79.2,-2.13,5.55,"Petit, Yusmeiro",670541,433589,NaN,ball,...,Standard,209.0,0.000,0.088,RP,0,1,5,2,1


In [10]:
sorted_relievers['closer_situation'] = ((sorted_relievers['pitch_number_appearance'] == 1) &
                                        (sorted_relievers['inning'] >= 9))

In [11]:
sorted_relievers['appearance'] = (sorted_relievers['pitch_number_appearance'] == 1)

In [12]:
# Group by pitcher and calculate the total outings and the number of closer situations
closers_df = sorted_relievers.groupby('player_name').agg(
    total_outings=('appearance', 'sum'),
    closer_situations=('closer_situation', 'sum')
).reset_index()

# Calculate the percentage of outings that were in closer situations
closers_df['closer_situation_percentage'] = (closers_df['closer_situations'] / closers_df['total_outings']) * 100

In [13]:
closers_df = closers_df[(closers_df['closer_situation_percentage'] >= 40) & (closers_df['closer_situations'] >= 40)]
closers_df.sort_values(by=['closer_situation_percentage'], ascending=False)
# let's set a threshold of 40% closer_situation percentage

,player_name,total_outings,closer_situations,closer_situation_percentage
873,"Pressly, Ryan",179,173,96.648045
441,"Hader, Josh",177,170,96.045198
722,"Melancon, Mark",126,112,88.888889
533,"Jansen, Kenley",185,164,88.648649
207,"Clase, Emmanuel",223,197,88.340807
521,"Iglesias, Raisel",190,148,77.894737
472,"Hendriks, Liam",132,102,77.272727
942,"Romano, Jordan",184,141,76.630435
297,"Díaz, Edwin",124,93,75.000000
575,"Kimbrel, Craig",197,146,74.111675


In [14]:
# Function to preprocess DataFrame
def preprocess_df(df):
    df['Name'] = df['Name'].str.replace(r'\*$', '', regex=True)
    multiple_entries = df[df.duplicated(subset=['Name', 'Age'], keep=False)]
    has_tot = multiple_entries['Tm'] == 'TOT'
    pitchers_with_tot = multiple_entries[has_tot]['Name'].unique()
    filtered_df = df[(~df['Name'].isin(pitchers_with_tot)) | ((df['Name'].isin(pitchers_with_tot)) & (df['Tm'] == 'TOT'))]
    tot_rows = filtered_df[filtered_df['Tm'] == 'TOT']
    max_ip_indices = tot_rows.groupby(['Name', 'Age'])['IP'].idxmax()
    final_df = filtered_df[filtered_df['Tm'] != 'TOT']
    final_df = pd.concat([final_df, filtered_df.loc[max_ip_indices]], ignore_index=True)
    return final_df

In [15]:
def convert_ip(ip):
    # Split the innings pitched into whole and fractional parts
    whole, fraction = divmod(ip, 1)
    # Convert the fractional part to decimal (e.g., 0.2 -> 2/3)
    return whole + (fraction * 10 / 3)

In [16]:
# Load datasets
leverage_2021 = pd.read_excel("2021 leverage index.xlsx")
leverage_2022 = pd.read_excel("2022 leverage index.xlsx")
leverage_2023 = pd.read_excel("2023 leverage index.xlsx")

# Apply preprocessing to each DataFrame
leverage_2021 = preprocess_df(leverage_2021)
leverage_2022 = preprocess_df(leverage_2022)
leverage_2023 = preprocess_df(leverage_2023)

# Combine the DataFrames
combined_df = pd.concat([leverage_2021, leverage_2022, leverage_2023], ignore_index=True)

# Compute weighted average of aLI
grouped = combined_df.groupby('Name').apply(lambda x: np.average(x['aLI'], weights=x['IP']) if np.sum(x['IP']) > 0 else 0).reset_index(name='Weighted_aLI')

# Also, adding total IP for each pitcher
grouped['Total_IP'] = combined_df.groupby('Name')['IP'].sum().values

# Replace NA's with zero - This line is effectively redundant given the if-else condition above, but kept for clarity
grouped['Weighted_aLI'] = grouped['Weighted_aLI'].fillna(0)

# The final DataFrame now correctly contains the name, weighted average aLI, and total IP for each pitcher
final_df = grouped

# Define the bins for the ranges of weighted aLI
bins = [0, 0.8, 1.5, np.inf]

# Define the labels for each bin
labels = ['Low Leverage', 'Average Leverage', 'High Leverage']

# Categorize weighted aLI into the defined bins
final_df['Leverage Category'] = pd.cut(final_df['Weighted_aLI'], bins=bins, labels=labels, include_lowest=True)

In [18]:
final_df[final_df['Total_IP'] >= 40].sort_values(by=['Weighted_aLI'], ascending=False)

,Name,Weighted_aLI,Total_IP,Leverage Category
354,Devin Williams,2.259135,172.4,High Leverage
633,Jordan Romano,2.133516,186.0,High Leverage
335,David Robertson,2.080972,140.3,High Leverage
44,Alexis Díaz,2.067877,130.3,High Leverage
711,Kenley Jansen,2.032009,177.2,High Leverage
...,...,...,...,...
691,Junior Guerra,0.435000,65.1,Low Leverage
262,Cole Sands,0.430479,51.4,Low Leverage
394,Edwin Uceta,0.424067,40.1,Low Leverage
1096,Stephen Nogosek,0.418060,50.2,Low Leverage


In [19]:
# Convert to excel to do vlookup with approximate matches
final_df.to_excel('final_df.xlsx', index=False)

In [20]:
# Load merged dataset
leverage_merge = pd.read_excel('merge_leverage.xlsx')
leverage_merge = leverage_merge.drop_duplicates(subset=['NameASCII'])

# Load fangraphs data
fangraphs_df = pd.read_csv('fangraphs_season_level.csv')

# Merge the DataFrames
merged_df = pd.merge(fangraphs_df, leverage_merge, left_on='NameASCII', right_on='NameASCII', how='left')

In [28]:
merged_df['Pitches']

0         67
1         92
2         15
3         69
4       1146
        ... 
3320     588
3321      76
3322    1898
3323     568
3324     583
Name: Pitches, Length: 3325, dtype: int64

In [37]:
merged_df

df_leverage_lab = merged_df.loc[merged_df.groupby('MLBAMID')['Pitches'].idxmax()].loc[:,["MLBAMID", "Leverage Category"]]

In [57]:
df_leverage_lab = df_leverage_lab.rename(columns={"Leverage Category":"leverage_label"})

In [41]:
#pitches = pd.read_csv('savant_pitch_level.csv')

# Sort relievers
relievers = pitches.loc[pitches['role_key'] == 'RP', :]
sorted_relievers = relievers.sort_values(by=['game_date', 'pitcher', 'pitch_number_appearance'])

# Mark the first pitch of each appearance
sorted_relievers['appearance'] = (sorted_relievers['pitch_number_appearance'] == 1)

print(sorted_relievers)

# Define inning categories
def inning_category(inning):
    if inning <= 5:
        return 'Early'
    elif inning <= 7:
        return 'Middle'
    else:
        return 'Late'

# Apply inning category based on the inning of the first pitch of each appearance
sorted_relievers['inning_category'] = sorted_relievers[sorted_relievers['appearance'] == True]['inning'].apply(inning_category)

# Fill NaN values for rows where 'appearance' is False
sorted_relievers['inning_category'] = sorted_relievers.groupby('player_name')['inning_category'].ffill()

# Group by pitcher and inning category to count appearances in each category
reliever_categories = sorted_relievers.groupby('player_name')['inning_category'].value_counts().unstack(fill_value=0)

# Identify the most frequent category for each pitcher
reliever_categories['Most_Frequent'] = reliever_categories.idxmax(axis=1)
reliever_categories.reset_index(inplace=True)


        pitch_type   game_date  release_speed  release_pos_x  release_pos_z  \
17863           FF  2021-04-01           87.5          -0.33           5.67   
17758           FF  2021-04-01           87.2          -0.38           5.52   
17615           FC  2021-04-01           83.5          -0.66           5.60   
17484           FF  2021-04-01           85.6          -2.25           5.52   
17322           CH  2021-04-01           79.2          -2.13           5.55   
...            ...         ...            ...            ...            ...   
2133022         FF  2023-10-01           97.7          -2.36           6.13   
2132994         FF  2023-10-01           98.0          -2.32           6.16   
2132955         SL  2023-10-01           85.7          -2.36           6.02   
2132921         FF  2023-10-01          100.0          -2.18           6.29   
2132882         SL  2023-10-01           87.3          -2.35           6.05   

             player_name  batter  pitcher     event

In [50]:
# Sort relievers
relievers = pitches.loc[pitches['role_key'] == 'RP', :]
sorted_relievers = relievers.sort_values(by=['game_date', 'pitcher', 'pitch_number_appearance'])

# Mark the first pitch of each appearance
sorted_relievers['appearance'] = sorted_relievers['pitch_number_appearance'] == 1

# Define inning categories
def inning_category(inning):
    if inning <= 5:
        return 'Early'
    elif inning <= 7:
        return 'Middle'
    else:
        return 'Late'

# Apply inning category for the first pitch of each appearance
sorted_relievers.loc[sorted_relievers['appearance'], 'inning_category'] = sorted_relievers[sorted_relievers['appearance']]['inning'].apply(inning_category)

# Ensure that all appearances have an 'inning_category', even if they're not the first pitch
# This is done by grouping by 'player_name' and then forward-filling the 'inning_category' within each group
sorted_relievers['inning_category'] = sorted_relievers.groupby(['game_date', 'player_name'])['inning_category'].ffill()

# Now, 'inning_category' should be correctly populated for all rows, not just the first pitch of each appearance
# Next, count the number of appearances in each category for each pitcher
reliever_categories = sorted_relievers.groupby('pitcher')['inning_category'].value_counts().unstack(fill_value=0)

# Identify the most frequent category for each pitcher
reliever_categories['Most_Frequent'] = reliever_categories.idxmax(axis=1)

reliever_categories.reset_index(inplace=True)

In [62]:
# Group by player_name and inning_category to sum appearances
reliever_summaries = sorted_relievers.groupby(['pitcher', 'inning_category'])['appearance'].sum().unstack(fill_value=0)

# Identify the most frequent category for each pitcher by finding the max across columns for each row
reliever_summaries['Most_Frequent'] = reliever_summaries.idxmax(axis=1)

# Reset index to flatten the DataFrame if needed
reliever_summaries = (
    reliever_summaries.reset_index().
    rename(columns={'Most_Frequent':'time_of_game_label'}).
    loc[:,["pitcher", "time_of_game_label"]])

reliever_summaries

inning_category,pitcher,time_of_game_label
0,405395,Late
1,424144,Late
2,425844,Early
3,425877,Late
4,429722,Late
...,...,...
1254,694813,Early
1255,695243,Middle
1256,696147,Late
1257,700363,Middle


In [63]:
labels = pd.merge(df_leverage_lab, reliever_summaries, left_on = 'MLBAMID', right_on='pitcher')

In [65]:
labels.to_csv('labels.csv')

In [25]:
# Convert to excel to do vlookup with approximate matches
reliever_summaries.to_excel('reliever_summaries.xlsx', index=False)
merged_df.to_excel('reliever_summaries.xlsx', index=False)

# # Load merged dataset
# leverage_merge = pd.read_excel('merge_leverage.xlsx')

# # Load fangraphs data
# fangraphs = pd.read_csv('fangraphs_season_level.csv')

# # Merge the DataFrames
# merged_df = pd.merge(fangraphs_df, leverage_merge, left_on='NameASCII', right_on='NameASCII', how='left')

In [ ]:
merged_df